In [1]:
import pandas as pd
df = pd.read_csv("inference_data_indices.csv")
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')

In [2]:
hazel = pd.read_csv("hazel.csv", delimiter='\t')
hazel = hazel[hazel["occurrenceStatus"] == "PRESENT"]
hazel = hazel[hazel['year']>=2022]
hazel['eventDate'] = pd.to_datetime(hazel['eventDate'], format='%Y-%m-%d', errors='coerce')
hazel = hazel[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
hazel = hazel.dropna().reset_index(drop=True)

birch = pd.read_csv("birch.csv", delimiter='\t')
birch = birch[birch["occurrenceStatus"] == "PRESENT"]
birch = birch[birch['year']>=2022]
birch['eventDate'] = pd.to_datetime(birch['eventDate'], format='%Y-%m-%d', errors='coerce')
birch = birch[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
birch = birch.dropna().reset_index(drop=True)

beech = pd.read_csv("beech.csv", delimiter='\t')
beech = beech[beech["occurrenceStatus"] == "PRESENT"]
beech = beech[beech['year']>=2022]
beech['eventDate'] = pd.to_datetime(beech['eventDate'], format='%Y-%m-%d', errors='coerce')
beech = beech[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
beech = beech.dropna().reset_index(drop=True)

honeysuckle = pd.read_csv("honeysuckle.csv", delimiter='\t')
honeysuckle = honeysuckle[honeysuckle["occurrenceStatus"] == "PRESENT"]
honeysuckle = honeysuckle[honeysuckle['year']>=2022]
honeysuckle['eventDate'] = pd.to_datetime(honeysuckle['eventDate'], format='%Y-%m-%d', errors='coerce')
honeysuckle = honeysuckle[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
honeysuckle = honeysuckle.dropna().reset_index(drop=True)

oak = pd.read_csv("oak.csv", delimiter='\t')
oak = oak[oak["occurrenceStatus"] == "PRESENT"]
oak = oak[oak['year']>=2022]
oak['eventDate'] = pd.to_datetime(oak['eventDate'], format='%Y-%m-%d', errors='coerce')
oak = oak[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
oak = oak.dropna().reset_index(drop=True)

hawthorn = pd.read_csv("hawthorn.csv", delimiter='\t')
hawthorn = hawthorn[hawthorn["occurrenceStatus"] == "PRESENT"]
hawthorn = hawthorn[hawthorn['year']>=2022]
hawthorn['eventDate'] = pd.to_datetime(hawthorn['eventDate'], format='%Y-%m-%d', errors='coerce')
hawthorn = hawthorn[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
hawthorn = hawthorn.dropna().reset_index(drop=True)

/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_79401/1357841717.py:1: DtypeWarning: Columns (10,38,39,41,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  hazel = pd.read_csv("hazel.csv", delimiter='\t')
/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_79401/1357841717.py:8: DtypeWarning: Columns (10,38,39,41,46) have mixed types. Specify dtype option on import or set low_memory=False.
  birch = pd.read_csv("birch.csv", delimiter='\t')
/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_79401/1357841717.py:15: DtypeWarning: Columns (10,39,41,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  beech = pd.read_csv("beech.csv", delimiter='\t')
/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_79401/1357841717.py:22: DtypeWarning: Columns (10,38,39,41,45,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  honeysuckle = pd.read_csv("honeysuckle.csv", delimiter

In [4]:
import numpy as np
from sklearn.neighbors import BallTree
from tqdm import tqdm
# Convert lat/lon to radians for BallTree
dormice_coords = np.deg2rad(df[['decimalLatitude', 'decimalLongitude']].values)
hazel_coords = np.deg2rad(hazel[['decimalLatitude', 'decimalLongitude']].values)
birch_coords = np.deg2rad(birch[['decimalLatitude', 'decimalLongitude']].values)
beech_coords = np.deg2rad(beech[['decimalLatitude', 'decimalLongitude']].values)
honeysuckle_coords = np.deg2rad(honeysuckle[['decimalLatitude', 'decimalLongitude']].values)
oak_coords = np.deg2rad(oak[['decimalLatitude', 'decimalLongitude']].values)
hawthorn_coords = np.deg2rad(hawthorn[['decimalLatitude', 'decimalLongitude']].values)


# Build BallTree using haversine metric
tree_hazel = BallTree(hazel_coords, metric='haversine')
tree_birch = BallTree(birch_coords, metric='haversine')
tree_beech = BallTree(beech_coords, metric='haversine')
tree_honeysuckle = BallTree(honeysuckle_coords, metric='haversine')
tree_oak = BallTree(oak_coords, metric='haversine')
tree_hawthorn = BallTree(hawthorn_coords, metric='haversine')

# Define search radius: 1 km in radians
earth_radius_km = 6371.0
radius = 1 / earth_radius_km  # 1 km in radians

# Initialize presence columns
df['hazel_presence'] = 0
df['birch_presence'] = 0
df['beech_presence'] = 0
df['honeysuckle_presence'] = 0
df['oak_presence'] = 0
df['hawthorn_presence'] = 0

# Iterate through each dormice point with tqdm for progress tracking
for i in tqdm(range(len(df))):
    point = dormice_coords[i].reshape(1, -1)
    event_month = df.loc[i, 'eventDate'].month
    event_year = df.loc[i, 'eventDate'].year

    # HAZEL
    idxs = tree_hazel.query_radius(point, r=radius)[0]
    for j in idxs:
        hazel_date = hazel.loc[j, 'eventDate']
        if hazel_date.month == event_month and hazel_date.year == event_year:
            df.at[i, 'hazel_presence'] = 1
            break  # Found at least one match, no need to check further

    # BIRCH
    idxs = tree_birch.query_radius(point, r=radius)[0]
    for j in idxs:
        birch_date = birch.loc[j, 'eventDate']
        if birch_date.month == event_month and birch_date.year == event_year:
            df.at[i, 'birch_presence'] = 1
            break
            
    # BEECH
    idxs = tree_beech.query_radius(point, r=radius)[0]
    for j in idxs:
        beech_date = beech.loc[j, 'eventDate']
        if beech_date.month == event_month and beech_date.year == event_year:
            df.at[i, 'beech_presence'] = 1
            break
            
    # HONEYSUCKLE
    idxs = tree_honeysuckle.query_radius(point, r=radius)[0]
    for j in idxs:
        honeysuckle_date = honeysuckle.loc[j, 'eventDate']
        if honeysuckle_date.month == event_month and honeysuckle_date.year == event_year:
            df.at[i, 'honeysuckle_presence'] = 1
            break
            
    # OAK
    idxs = tree_oak.query_radius(point, r=radius)[0]
    for j in idxs:
        oak_date = oak.loc[j, 'eventDate']
        if oak_date.month == event_month and oak_date.year == event_year:
            df.at[i, 'oak_presence'] = 1
            break
            
    # HAWTHORN
    idxs = tree_hawthorn.query_radius(point, r=radius)[0]
    for j in idxs:
        hawthorn_date = hawthorn.loc[j, 'eventDate']
        if hawthorn_date.month == event_month and hawthorn_date.year == event_year:
            df.at[i, 'hawthorn_presence'] = 1
            break
            
            
            
            


100%|██████████| 244359/244359 [00:41<00:00, 5872.49it/s]


In [5]:
from pyproj import Transformer
from rasterio.windows import Window
import rasterio

land_cover_map = {
    1: "Deciduous woodland",
    2: "Coniferous woodland",
    3: "Arable",
    4: "Improved grassland",
    5: "Neutral grassland",
    6: "Calcareous grassland",
    7: "Acid grassland",
    8: "Fen",
    9: "Heather",
    10: "Heather grassland",
    11: "Bog",
    12: "Inland rock",
    13: "Saltwater",
    14: "Freshwater",
    15: "Supralittoral rock",
    16: "Supralittoral sediment",
    17: "Littoral rock",
    18: "Littoral sediment",
    19: "Saltmarsh",
    20: "Urban",
    21: "Suburban"
}

# Batch coordinate transformation
transformer_ni = Transformer.from_crs("EPSG:4326", "EPSG:29903", always_xy=True)
transformer_gb = Transformer.from_crs("EPSG:4326", "EPSG:27700", always_xy=True)

coords = list(zip(df['decimalLongitude'], df['decimalLatitude']))
df['easting_ni'], df['northing_ni'] = zip(*transformer_ni.itransform(coords))
df['easting_gb'], df['northing_gb'] = zip(*transformer_gb.itransform(coords))

# Raster processing optimization
gb_raster = 'gblcm2023_10m.tif'
n_ireland_raster = 'nilcm2023_10m.tif'


def get_land_cover_class(row):
    try:
        # Try GB raster first
        with rasterio.open(gb_raster) as src:
            row_idx, col_idx = src.index(row['easting_gb'], row['northing_gb'])
            # Read a small window around the point for better performance
            window = Window(col_idx, row_idx, 1, 1)
            land_cover_class = src.read(1, window=window)[0, 0]

            if land_cover_class == 0:  # Check NI raster if GB is 0
                with rasterio.open(n_ireland_raster) as src_ni:
                    row_idx, col_idx = src_ni.index(row['easting_ni'], row['northing_ni'])
                    window = Window(col_idx, row_idx, 1, 1)
                    land_cover_class = src_ni.read(1, window=window)[0, 0]

        return land_cover_map.get(land_cover_class, "Unknown")
    except Exception as e:
        print(f"Error processing row: {e}")
        return "Unknown"


tqdm.pandas()
df['Land_cover'] = df.progress_apply(get_land_cover_class, axis=1)

  0%|          | 1121/244359 [00:02<08:49, 459.65it/s]

Error processing row: index 0 is out of bounds for axis 0 with size 0


 19%|█▉        | 46166/244359 [02:05<08:36, 383.44it/s]

Error processing row: index 0 is out of bounds for axis 0 with size 0


 32%|███▏      | 79265/244359 [03:38<06:10, 445.22it/s]

...

 98%|█████████▊| 239105/244359 [11:46<00:13, 389.24it/s]

Error processing row: index 0 is out of bounds for axis 0 with size 0


 99%|█████████▉| 242766/244359 [11:54<00:03, 464.23it/s]

Error processing row: index 0 is out of bounds for axis 0 with size 0
Error processing row: index 0 is out of bounds for axis 0 with size 0


100%|██████████| 244359/244359 [11:58<00:00, 340.18it/s]


In [6]:
df = df.dropna()
df=df.drop(['easting_ni', 'northing_ni', 'easting_gb', 'northing_gb'],axis=1)
# Remove where Land_cover is Unknown
df = df[df['Land_cover'] != "Unknown"].reset_index(drop=True)
df.to_csv("dormice_inference_data_preprocessed.csv", index=False)

In [8]:
df = pd.read_csv("dormice_inference_data_preprocessed.csv")
df = pd.get_dummies(df, columns=['Land_cover'])
df = df.astype({col: 'int' for col in df.select_dtypes(include='bool').columns})

#read scaler
import joblib
scaler = joblib.load("dormice_scaler.pkl")
# Scale the data
env_vars = ['BSI', 'LST', 'MNDWI', 'NDBI', 'NDSI', 'NDVI', 'NDWI', 'SAVI', 'UI']
df[env_vars] = scaler.transform(df[env_vars])
# drop Unnamed: 0
# df = df.drop(columns=['Unnamed: 0'])
df

,decimalLongitude,decimalLatitude,eventDate,BSI,LST,MNDWI,NDBI,NDSI,NDVI,NDWI,...,Land_cover_Inland rock,Land_cover_Littoral rock,Land_cover_Littoral sediment,Land_cover_Neutral grassland,Land_cover_Saltmarsh,Land_cover_Saltwater,Land_cover_Suburban,Land_cover_Supralittoral rock,Land_cover_Supralittoral sediment,Land_cover_Urban
0,-6.347425,49.893656,2025-06-01,0.293274,-0.002672,0.412461,-0.007846,0.412461,-0.165498,0.304107,...,0,0,0,0,0,0,1,0,0,0
1,-6.321265,49.912633,2025-06-01,0.463918,-1.438091,0.627304,-0.053729,0.627304,-0.336127,0.564584,...,0,0,0,0,0,0,0,0,0,0
2,-6.307377,49.913150,2025-06-01,-0.255497,0.000182,-0.060399,-0.375857,-0.060399,0.340652,-0.252897,...,0,0,0,0,0,0,1,0,0,0
3,-6.293488,49.913666,2025-06-01,-0.653921,0.105083,-0.376596,-0.728743,-0.376596,0.771306,-0.703132,...,0,0,0,0,0,0,0,0,0,0
4,-6.308177,49.922119,2025-06-01,-0.258903,-0.011762,0.201248,-0.992148,0.201248,0.446479,-0.159607,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243512,-0.808766,60.824609,2025-06-01,-0.578236,-0.911807,-0.920890,0.676035,-0.920890,0.389105,-0.687037,...,0,0,0,0,0,0,0,0,0,0
243513,-0.790383,60.824445,2025-06-01,1.006828,-0.634710,0.967822,1.152616,0.967822,-0.813325,1.554708,...,0,0,0,0,0,0,0,0,0,0
243514,-0.881988,60.834217,2025-06-01,-0.437808,-0.809255,-0.662704,0.449416,-0.662704,0.327916,-0.525695,...,0,0,0,0,0,0,0,0,0,0
243515,-0.845210,60.833906,2025-06-01,0.570275,-0.952518,0.196350,1.553755,0.196350,-0.792752,0.936717,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df.to_csv("dormice_inference_data_preprocessed_final.csv", index=False)

In [11]:
df = pd.read_csv("dormice_inference_data_preprocessed_final.csv")
#read model from brown_hare_model.pkl
import joblib
model = joblib.load("dormice_model.pkl")
# predict_proba
probabilities = model.predict_proba(df.drop(columns=['decimalLatitude', 'decimalLongitude', 'eventDate']))
# add probabilities to df
df['probability'] = probabilities[:, 1]  # Assuming the second column is the positive class
df.to_csv("dormice_results.csv", index=False)
df

,decimalLongitude,decimalLatitude,eventDate,BSI,LST,MNDWI,NDBI,NDSI,NDVI,NDWI,...,Land_cover_Littoral rock,Land_cover_Littoral sediment,Land_cover_Neutral grassland,Land_cover_Saltmarsh,Land_cover_Saltwater,Land_cover_Suburban,Land_cover_Supralittoral rock,Land_cover_Supralittoral sediment,Land_cover_Urban,probability
0,-6.347425,49.893656,2025-06-01,0.293274,-0.002672,0.412461,-0.007846,0.412461,-0.165498,0.304107,...,0,0,0,0,0,1,0,0,0,0.269519
1,-6.321265,49.912633,2025-06-01,0.463918,-1.438091,0.627304,-0.053729,0.627304,-0.336127,0.564584,...,0,0,0,0,0,0,0,0,0,0.448344
2,-6.307377,49.913150,2025-06-01,-0.255497,0.000182,-0.060399,-0.375857,-0.060399,0.340652,-0.252897,...,0,0,0,0,0,1,0,0,0,0.314562
3,-6.293488,49.913666,2025-06-01,-0.653921,0.105083,-0.376596,-0.728743,-0.376596,0.771306,-0.703132,...,0,0,0,0,0,0,0,0,0,0.612908
4,-6.308177,49.922119,2025-06-01,-0.258903,-0.011762,0.201248,-0.992148,0.201248,0.446479,-0.159607,...,0,0,0,0,0,0,0,0,1,0.355127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243512,-0.808766,60.824609,2025-06-01,-0.578236,-0.911807,-0.920890,0.676035,-0.920890,0.389105,-0.687037,...,0,0,0,0,0,0,0,0,0,0.448477
243513,-0.790383,60.824445,2025-06-01,1.006828,-0.634710,0.967822,1.152616,0.967822,-0.813325,1.554708,...,0,0,0,0,0,0,0,0,0,0.423370
243514,-0.881988,60.834217,2025-06-01,-0.437808,-0.809255,-0.662704,0.449416,-0.662704,0.327916,-0.525695,...,0,0,0,0,0,0,0,0,0,0.466217
243515,-0.845210,60.833906,2025-06-01,0.570275,-0.952518,0.196350,1.553755,0.196350,-0.792752,0.936717,...,0,0,0,0,0,0,0,0,0,0.373095
